# Exploratory Data Analysis for Click-Through Rate Prediction

The dataset was provided by the mobile advertising company Avazu as part of a [Kaggle competition](https://www.kaggle.com/competitions/avazu-ctr-prediction/overview)

*Dataset Citation: Steve Wang, Will Cukierski. (2014). Click-Through Rate Prediction. Kaggle. https://kaggle.com/competitions/avazu-ctr-prediction*


# Data Preparation

## Create new transformed features on test and training data

Let's reload the data to start afresh.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
contentRoot = '/content/drive/MyDrive/Github/machinelearning/click-through-prediction'

In [ ]:
train_df = pd.read_csv('{}/data/train.gz'.format(contentRoot))

In [ ]:
print('Train data: {}'.format(str(train_df.shape)))

Train data: (40428967, 24)


### Add a new hour column

In [ ]:
train_df['datetime'] = pd.to_datetime(train_df['hour'],format = '%y%m%d%H')

In [ ]:
train_df['hour'] = train_df['datetime'].dt.strftime('%H')

### Add a new day_of_week column

In [ ]:
train_df['day_of_week'] = train_df['datetime'].dt.day_name()

### Reduce cardinality of columns

We will reduce the cardinality of `device_model`, `app_id`, `site_id`, `site_domain`, and `C14` by retaining unique values that represent a bulk of the values and mapping the rest to `Others`

In [ ]:
def filter_to_frequent_values_gt_threshold(train_data, column, threshold):
  series = train_data[column]
  series_df = pd.DataFrame(series.value_counts().sort_values(ascending=False))
  series_df.columns = ['value_count']
  series_df['cum_perc'] =  100*series_df['value_count'].cumsum()/series_df['value_count'].sum()
  series_df = series_df.where(series_df['cum_perc'] < threshold).dropna()
  series_df['target_'+column] = series_df.index
  series_df.drop(['value_count', 'cum_perc'], axis=1)
  train_data[column] = train_data.join(other=series_df, on=column, how='left')['target_'+column]

In [ ]:
filter_to_frequent_values_gt_threshold(train_df, 'device_model', 95)

In [ ]:
filter_to_frequent_values_gt_threshold(train_df, 'app_id', 99)

In [ ]:
filter_to_frequent_values_gt_threshold(train_df, 'site_id', 99)

In [ ]:
filter_to_frequent_values_gt_threshold(train_df, 'site_domain', 99)

In [ ]:
train_df = train_df.fillna('other')

In [ ]:
filter_to_frequent_values_gt_threshold(train_df, 'C14', 99)

In [ ]:
train_df = train_df.fillna(-1)

In [ ]:
print('Number of `other` values (out of {}) in'.format(train_df.shape[0]))
print('device_model: {}'.format(train_df['device_model'].value_counts()['other']))
print('app_id: {}'.format(train_df['app_id'].value_counts()['other']))
print('site_id: {}'.format(train_df['site_id'].value_counts()['other']))
print('site_domain: {}'.format(train_df['site_domain'].value_counts()['other']))
print('C14: {}'.format(train_df['C14'].value_counts()[-1]))

Number of `other` values (out of 40428967) in
device_model: 2022500
app_id: 404942
site_id: 404841
site_domain: 404849
C14: 404899


### Write the prepared data back to a file

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40428967 entries, 0 to 40428966
Data columns (total 26 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                float64       
 1   click             int64         
 2   hour              object        
 3   C1                int64         
 4   banner_pos        int64         
 5   site_id           object        
 6   site_domain       object        
 7   site_category     object        
 8   app_id            object        
 9   app_domain        object        
 10  app_category      object        
 11  device_id         object        
 12  device_ip         object        
 13  device_model      object        
 14  device_type       int64         
 15  device_conn_type  int64         
 16  C14               float64       
 17  C15               int64         
 18  C16               int64         
 19  C17               int64         
 20  C18               int64         
 21  C19   

In [ ]:
train_df.drop(columns=['id', 'datetime', 'device_ip', 'device_id'], axis=1)

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,day_of_week
0,0,00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,2,15706.0,320,50,1722,0,35,-1,79,Tuesday
1,0,00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,15704.0,320,50,1722,0,35,100084,79,Tuesday
2,0,00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,15704.0,320,50,1722,0,35,100084,79,Tuesday
3,0,00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,15706.0,320,50,1722,0,35,100084,79,Tuesday
4,0,00,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,...,0,18993.0,320,50,2161,0,35,-1,157,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40428962,1,23,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,...,0,17262.0,320,50,1872,3,39,100173,23,Thursday
40428963,0,23,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,...,2,23160.0,320,50,2667,0,47,-1,221,Thursday
40428964,0,23,1005,1,f61eaaae,6b59f079,f028772b,ecad2386,7801e8d9,07d7df22,...,0,20969.0,320,50,2372,0,813,-1,46,Thursday
40428965,1,23,1005,0,85f751fd,c4e18dd6,50e219e0,3c4b944d,2347f47a,0f2161f8,...,0,16859.0,320,50,1887,3,39,100194,23,Thursday


# Sample the dataframe and write to new files for model training

In [ ]:
sample_25 = train_df.sample(frac=0.25)

# Write the sample to a file
sample_25.to_csv(f"{contentRoot}/data/sample_25_prepped_data.csv.gz",
              compression='gzip',
              index=False)

In [ ]:
sample_10 = train_df.sample(frac=0.10)

# Write the sample to a file
sample_10.to_csv(f"{contentRoot}/data/sample_10_prepped_data.csv.gz",
              compression='gzip',
              index=False)